使用朴素贝叶斯过滤垃圾邮件

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import log
%matplotlib inline

使用python来文本分类

1.准备数据：从文中构建词向量

In [2]:
def loadDataSet():#来生成数据：文本列表和标签分类
    #文本集：每个[]是一个文本，里面是分割开的单词
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    #0-非侮辱性，1-侮辱性
    classVec = [0,1,0,1,0,1]    
    return postingList,classVec

def createVocabList(dataset):#获取词汇表：把文本集装换成一个不含重复词的列表
    vocabSet = set([])#创建一个空集,可以进行集合操作
    for document in dataset:
        vocabSet = vocabSet |set(document)#每次把文件中的不重复元素放入
    return list(vocabSet)

def setOfWords2Vec(vocabList, inputSet):#转化成向量:参数是词汇表和某个文档
    returnVec = [0]*len(vocabList)#构建vocabList长度的0向量
    for word in inputSet:  #把该文档中在词汇表中有该单词的位置标位1（one-hot向量）
        if word in vocabList:
            returnVec[vocabList.index(word)]=1 
        else:
             print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

2.训练算法：从词向量计算概率

In [3]:
import numpy as np
"""
函数说明:朴素贝叶斯分类器训练函数

Parameters:
    trainMatrix - 文档矩阵
    trainCategory - 每篇文档类别标签所构成的向量
Returns:
    p0Vect - 非侮辱类的条件概率数组
    p1Vect - 侮辱类的条件概率数组
    pAbusive - 文档属于侮辱类的概率
"""
def trainNB0(trainMatrix, trainGategory):
    numTrainDocs = len(trainMatrix)   #返回训练的文档数目
    numWords = len(trainMatrix[0])   #返回每一篇文档的词条数
    pAbusive = sum(trainGategory)/float(numTrainDocs)  #文档属于侮辱类的概率
    #用拉普拉斯修正”来平滑数据
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)  #创建numpy.zeros数组,词条出现数初始化为0
    p0Denom = 2.0; p1Denom = 2.0   #分母初始化为2
    for i in range(numTrainDocs):
        if trainGategory[i] == 1:  #统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:  #统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)#log防止下溢出
    p0Vect = np.log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive  #返回属于非侮辱类的条件概率数组，属于侮辱类的条件概率数组，文档属于侮辱类的概率

3.测试算法：根据现实情况修改分类器

In [4]:
"""
函数说明:朴素贝叶斯分类函数

Parameters:
    vec2Classify - 待分类的词条数组
    p0Vec - 非侮辱类的条件概率数组
    p1Vec - 侮辱类的条件概率数组
    pClass1 - 文档属于侮辱类的概率
Returns:
    类别标签 -   1或0
"""
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify*p1Vec)+np.log(pClass1)# #对应元素相乘。logA * B = logA + logB，所以这里加上log(pClass1)
    p0 = sum(vec2Classify*p0Vec)+np.log(1.0-pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(np.array(trainMat),np.array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0V,p1V,pAb))

4.准备数据：文档词袋模型:和setOfWords2Vec区别是每个单词可以重复多次

In [5]:
#词袋模型
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

示例：使用朴素贝叶斯过滤垃圾邮件

In [6]:
def textParse(bigString):
    """
    输入很长的字符串，转换为向量
    参数：
        bigString -- 长字符串
    返回：
        去掉少于两个字符，转换为小写的字符串
    """
    import re
    listOfTokens = re.split(r'\W*', bigString)#正则表达式来去掉空格和标点符号
    return [tok.lower() for tok in listOfTokens if len(tok) > 2] #全变成小写，去掉少于两个字符的字符串

def spamTest():
    #新建三个列表
    docList=[]; classList = []; fullText =[]
    #遍历垃圾邮件和正常邮件，各25个
    for i in range(1,26):
        #读取垃圾邮件
        wordList = textParse(open("email/spam/{}.txt".format(i), errors = 'ignore').read())
        #添加到列表
        docList.append(wordList)
        fullText.extend(wordList)
        #添加到类
        classList.append(1)
        #读取正常邮件
        #ham中的23.txt总是报错有不能解读的字节，选择忽略该错误
        wordList = textParse(open("email/ham/{}.txt".format(i), errors = 'ignore').read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    #创建词汇表
    vocabList = createVocabList(docList)
    #训练集和测试集序号集
    trainingSet = list(range(50)); testSet=[]
    #随机抽取训练集中的10个序号，放入测试集
    for i in range(10):
        #生成随机序号
        randIndex = np.int(np.random.uniform(0,len(trainingSet)))
        #序号对应的元素由训练集移动到测试集中
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex]) 
    #新建训练矩阵和训练标签
    trainMat=[]; trainClasses = []
    #对于训练集中的元素
    for docIndex in trainingSet:
        #对应词袋添加到训练矩阵中
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        #类别添加到标签中
        trainClasses.append(classList[docIndex])
    #训练朴素贝叶斯分类器
    p0V,p1V,pSpam = trainNB0(np.array(trainMat),np.array(trainClasses))
    #错误计数器初始化为0
    errorCount = 0
    #对于测试集
    for docIndex in testSet:
        #得到词袋向量
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        #判断结果
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            #统计错误
            errorCount += 1
            #打印错误信息
            print("错误序号为：{}".format(docList[docIndex]))
    print("总准确率为：{}".format(1 - np.float(errorCount)/len(testSet)))

In [7]:
spamTest()

错误序号为：['home', 'based', 'business', 'opportunity', 'knocking', 'your', 'door', 'don抰', 'rude', 'and', 'let', 'this', 'chance', 'you', 'can', 'earn', 'great', 'income', 'and', 'find', 'your', 'financial', 'life', 'transformed', 'learn', 'more', 'here', 'your', 'success', 'work', 'from', 'home', 'finder', 'experts']
总准确率为：0.9


D:\Anaconda3\lib\re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
